In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import seaborn as sns

## Load the training data into feature matrix, class labels, and event ids, and separate the dataset:

In [2]:
from implementations import *

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


In [4]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [63]:
logistic = False
tX_list, ids_list, y_list = separate_dataset(tX, ids, y, logistic)
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test) 

In [64]:
tX_list, tX_test_list, y_list = clean_data(tX_list, tX_test_list, y_list)

## GRID SEARCH RIDGE REGRESSION

In [66]:
function = least_squares_GD

degrees = range(1,5)
lambdas = np.logspace(-6, 0, 1)
gammas = np.logspace(-6, 0, 3)

degree_vec = []
lambda_vec = []
gamma_vec = []

for i in range(6):
    print('Dataset part {l}:'.format(l = i))
    rmse_te, BestDeg, BestLambda, BestGamma = grid_search(y_list[i], tX_list[i], function, 4, degrees, lambdas, gammas, dataset = i)
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    gamma_vec.append(BestGamma)
    print('     Best degree: {d}'.format(d = BestDeg))
    print('     Best lambda: {m}'.format(m = BestLambda))
    print('     Best gamma: {m}'.format(m = BestGamma))
    print('     Loss: {lo}'.format(lo = rmse_te))

print(degree_vec)
print(lambda_vec)
print(gamma_vec)

Dataset part 0:


/Users/theopatron/Documents/EPFL/MA3/ML/ml-project-1-pml/scripts/implementations.py:10: RuntimeWarning: overflow encountered in square
  loss = ((y - tx.dot(w))**2).sum()/(2*len(y))   #MSE
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


loss error
loss error
loss error
loss error
loss error
loss error
loss error
loss error
loss error
loss error
loss error
loss error
loss error
loss error
loss error
loss error
     Best degree: 2
     Best lambda: 1e-06
     Best gamma: 0.001
     Loss: 6150.25
Dataset part 1:
loss error
loss error
loss error
loss error


KeyboardInterrupt: 

In [61]:
weights_list = []
loss_list = []
mat_tX_test_list = []
max_iters = 300

for i in range(6):
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], tX_test_list[i], i)
    # initial_w = np.zeros((mat_tX.shape[1],1))
    #w, l = ridge_regression(y_list[i], mat_tX, lambda_vec[i])
    #w, l = least_squares(y_list[i], mat_tX)
    initial_w = np.zeros(mat_tX.shape[1])
    w, l = least_squares_GD(y_list[i], mat_tX, initial_w, max_iters, gamma_vec[i])
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)

In [62]:
# Eval

y_pred_list = separated_eval(weights_list, mat_tX_test_list, logistic) 

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3], y_pred_list[4], y_pred_list[5]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3], ids_test_list[4], ids_test_list[5]))

OUTPUT_PATH = 'result.csv'
create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)